In [2]:
!pip install opencv-python

In [3]:
import cv2
import numpy as np

# 初始化攝像頭
cap = cv2.VideoCapture(0)

# 讀取背景影像
ret, background = cap.read()

# 轉為灰度影像
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

# 設定門檻值
threshold = 50

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 轉為灰度影像
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 背景相減
    diff = cv2.absdiff(background, gray_frame)
    
    _, diff = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # 顯示結果
    cv2.imshow('Background Subtraction', diff)

    # 檢查是否有按鍵按下
    k = cv2.waitKey(30) & 0xff
    if k == 27:  # 按下 'ESC' 鍵退出
        break

# 釋放攝像頭並關閉所有窗口
cap.release()
cv2.destroyAllWindows()
